### Microfinance Performance Indicators

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/bekay/Documents/Studies/MSc Financial Engineering/Models/credit_risk_portfolio.csv')

In [3]:
df.head()

,Loan_ID,Country,Loan_Amount,Tenor_Months,Interest_Rate,Borrower_Type,Secured,Days_Past_Due,Credit_Score,Repayment_History,...,Score_Factor,Delinquency_Factor,History_Factor,Collateral_Factor,PD,LGD_Base,LGD,EAD,Utilization,ECL
0,MF0001,Haiti,10502.13,12,0.1781,Agricultural,False,0,756.0,0.560,...,0.697143,1.0,1.440,1.0,0.050194,0.45,0.540,918.9390,0.083334,24.907763
1,MF0002,"Yemen, Republic",7301.87,42,0.1393,SME,False,0,743.0,0.861,...,0.734286,1.0,1.139,1.0,0.041818,0.45,0.540,547.6380,0.071428,12.366481
2,MF0003,Russia,8756.90,18,0.1071,Women-owned,False,0,423.0,0.885,...,1.648571,1.0,1.115,1.0,0.091908,0.45,0.540,6129.8265,0.666666,304.224778
3,MF0004,Libya,13044.55,60,0.1828,Agricultural,True,0,750.0,0.764,...,0.714286,1.0,1.236,0.8,0.035314,0.45,0.315,3424.1970,0.250000,38.090767
4,MF0005,Iran,4210.87,12,0.1341,Agricultural,False,0,521.0,0.879,...,1.368571,1.0,1.121,1.0,0.076708,0.45,0.540,2210.7120,0.500001,91.573332


In [4]:
df.columns

Index(['Loan_ID', 'Country', 'Loan_Amount', 'Tenor_Months', 'Interest_Rate',
       'Borrower_Type', 'Secured', 'Days_Past_Due', 'Credit_Score',
       'Repayment_History', 'Months_Elapsed', 'Outstanding_Balance',
       'Moody_Rating', 'Default_Spread', 'Country_Risk_Premium', 'PD_Base',
       'Score_Factor', 'Delinquency_Factor', 'History_Factor',
       'Collateral_Factor', 'PD', 'LGD_Base', 'LGD', 'EAD', 'Utilization',
       'ECL'],
      dtype='object')

In [5]:
print("\n" + "="*80)
print("Indicator 1: PORTFOLIO AT RISK (PAR  > 30 Days)")
print("="*80)


Indicator 1: PORTFOLIO AT RISK (PAR  > 30 Days)


Measures overdue loans as a percentage of the gross loan portfolio; critical for liquidity and risk reports, identifying economic/political vulnerabilities.

$\text{PAR} = \frac{\sum \text{Outstanding\_Balance (where Days\_Past\_Due > 30)}}{\sum \text{Outstanding\_Balance}} \times 100$ 
 where summation $\sum$ aggregates over qualifying loans.

In [6]:
par_loans = df[df['Days_Past_Due']> 30]
par_numerator = par_loans['Outstanding_Balance'].sum()
par_denominator = df['Outstanding_Balance'].sum()
par = par_numerator / par_denominator * 100
print(f'Portfolio at Risk (>30 days): {par:2f}%')

Portfolio at Risk (>30 days): 7.418913%


Portfolio at Risk (PAR) by Borrower Type

In [7]:
def compute_par(group):
    overdue_balance = group.loc[group['Days_Past_Due'] > 30, 'Outstanding_Balance'].sum()
    total_balance = group['Outstanding_Balance'].sum() 
    par = (overdue_balance / total_balance * 100) if total_balance > 0 else 0
    return par

In [8]:
par_by_type = df.groupby('Borrower_Type').apply(compute_par, include_groups=False).round(2)

In [9]:
par_df = pd.DataFrame({'Borrower_Type': par_by_type.index, 'PAR (%)': par_by_type.values})

In [10]:
par_df

,Borrower_Type,PAR (%)
0,Agricultural,13.85
1,Individual,3.83
2,SME,7.35
3,Women-owned,4.32


In [11]:
print('\n' + '='*80)
print('Indicator 2: Expected Credit Loss (ECL)')
print('='*80)


Indicator 2: Expected Credit Loss (ECL)


Forward-looking estimate of losses; supports profitability and capital utilization insights in impact funds.

$\text{ECL} = \text{PD} \times \text{LGD} \times \text{EAD}$ 
 where $\text{LGD}$ is Loss Given Default (0-1 scale), $\text{EAD}$ is Exposure at Default (e.g., Outstanding_Balance).

Group by Country for average ECL (portfolio risk insight)

In [12]:
ecl_by_country = df.groupby('Country').agg({'ECL':'mean'}).round(2)
ecl_by_country.head()

,ECL
Country,
Algeria,90.34
Brunei,246.80
Gambia,259.04
Guinea,297.15
Guinea-Bissau,254.57


### Write-Off Ratio Calculation in Microfinance Portfolio Analysis

In the context of microfinance portfolio management, the Write-Off Ratio represents the proportion of the loan portfolio deemed irrecoverable and removed from active assets, typically due to prolonged delinquency. This indicator is a key measure of credit quality and historical performance, reflecting the ultimate impact of defaults on profitability and capital reserves. Calculating the Write-Off Ratio supports in-depth financial modeling and reporting, enabling insights into risk patterns across portfolio dimensions such as borrower types. It aids in assessing the effectiveness of mitigation strategies, like enhanced due diligence for high-risk segments, in emerging market private debt funds.

The Write-Off Ratio is computed using a predefined threshold for irrecoverability, commonly set at loans overdue by more than 180 days (PAR > 180 days), as per industry standards from sources like the World Bank's MIX Market framework. The formula, applied at the subgroup level (e.g., by Borrower_Type), is:

$$\text{Write-Off Ratio} = \left( \frac{\sum \text{Loan\_Amount (where Days\_Past\_Due} > 180)}{\sum \text{Loan\_Amount}} \right) \times 100$$
where:

- $\sum \text{Loan\_Amount (where Days\_Past\_Due} > 180)$: The summation, denoted by $\sum$, aggregates the original loan amounts for loans exceeding the 180-day delinquency threshold.
- $\sum \text{Loan\_Amount}$: The total summation of original loan amounts across all loans in the subgroup, serving as the denominator to express the ratio as a percentage of the portfolio's initial exposure.


This metric differs from Portfolio at Risk (PAR), which tracks current delinquencies, by focusing on realized losses written off the balance sheet.

#### Results: Write-Off Ratio by Borrower Type
Utilizing the "credit_risk_portfolio.csv" dataset (500 loan records from emerging markets), the computation yields the following values, rounded to two decimal places:

| Borrower_Type | Write-Off Ratio (%) |
|---------------|---------------------|
| Agricultural  | 0.00                |
| Individual    | 0.00                |
| SME           | 0.00                |
| Women-owned   | 0.00                |

**Interpretation**: Across all borrower types, the Write-Off Ratio is 0.00%, indicating that no loans in this dataset meet the >180-day delinquency threshold for write-off. This suggests a resilient portfolio with effective early-stage risk controls, such as timely interventions or conservative lending criteria, which is favorable. In a real-world scenario, a non-zero ratio might highlight vulnerabilities (e.g., in agricultural loans due to seasonal risks), prompting scenario analyses for capital provisioning. If the dataset's delinquency levels are low overall, consider sensitivity testing with a lower threshold (e.g., >120 days) for forward-looking projections.




In [13]:
# Define a function to compute the Write-Off Ratio for each subgroup.

def compute_writeoff(group):
    
    overdue_amount = group.loc[group['Days_Past_Due'] > 180, 'Loan_Amount'].sum()  
    total_amount = group['Loan_Amount'].sum()  # Computes the sum for the full subgroup, ensuring the ratio reflects initial portfolio size.
    ratio = (overdue_amount / total_amount * 100) if total_amount > 0 else 0  # Scalar operations: Division and multiplication derive the percentage; the conditional (if-else) safeguards against division by zero in subgroups with no loans.
    return ratio  # The function yields a float representing the percentage.


# .apply(compute_writeoff, include_groups=False): Executes the function on each subgroup; include_groups=False (a boolean argument) omits the grouping column from the input to the function, adhering to future pandas standards and suppressing deprecation warnings.
writeoff_by_type = df.groupby('Borrower_Type').apply(compute_writeoff, include_groups=False).round(2)  # .round(2): A method applied to the resulting Series (a one-dimensional structure with .index for labels and .values for data) that rounds floats to two decimal places for concise reporting.

# Create an output DataFrame for tabular visualization.
writeoff_df = pd.DataFrame({'Borrower_Type': writeoff_by_type.index, 'Write-Off Ratio (%)': writeoff_by_type.values})  
print(writeoff_df.to_string(index=False))  

Borrower_Type  Write-Off Ratio (%)
 Agricultural                  0.0
   Individual                  0.0
          SME                  0.0
  Women-owned                  0.0


### Portfolio Yield and Probability of Default in Microfinance Analysis

In microfinance, Portfolio Yield quantifies the annualized income generated from interest and fees as a percentage of the average gross loan portfolio, serving as a primary indicator of financial sustainability and profitability. It reflects the effective return on deployed capital, adjusted for utilization levels, and is essential for fund models to evaluate performance alongside impact metrics. Conversely, Probability of Default (PD) measures the estimated likelihood that a borrower will fail to meet repayment obligations within a specified period, typically expressed as a decimal (0 to 1) or percentage. PD informs risk-adjusted forecasting and capital provisioning, aligning with the Portfolio Analyst's responsibilities in analyzing patterns, risks, and performance for strategic decision-making in emerging market private debt.

For this analysis, computations are segmented by Borrower_Type (Agricultural, Individual, SME, Women-owned) using the "credit_risk_portfolio.csv" dataset (500 loan records). The Portfolio Yield approximates effective yield on outstanding balances, weighted by utilization. PD is averaged directly from the pre-computed column, which incorporates base probabilities scaled by credit, delinquency, history, and collateral factors.

#### Formulas
The Portfolio Yield is calculated as:
$$\text{Portfolio Yield} = \left( \frac{\sum (\text{Interest\_Rate} \times \text{Utilization} \times \text{Outstanding\_Balance})}{\sum \text{Outstanding\_Balance}} \right) \times 100$$
where:

- $\sum (\text{Interest\_Rate} \times \text{Utilization} \times \text{Outstanding\_Balance})$: The summation, denoted by $\sum$, aggregates the interest accrued across loans, with Interest_Rate as the annual rate (decimal), Utilization as the drawn-down proportion (0-1 scale), and Outstanding_Balance as the current principal exposure.
- $\sum \text{Outstanding\_Balance}$: The total summation of outstanding balances in the subgroup, normalizing the yield to the active portfolio size.
The result is a percentage, representing weighted average yield.

The Average PD is the arithmetic mean of individual PD values:
$$\text{Average PD} = \frac{\sum \text{PD}}{n}$$
where $\sum \text{PD}$ is the summation of PD values over $n$ loans in the subgroup, and PD is the per-loan default probability (0-1 scale).

#### Results: Portfolio Yield and Average PD by Borrower Type
The following table presents the computed metrics, rounded for reporting precision:

| Borrower_Type | Portfolio Yield (%) | Average PD |
|---------------|---------------------|------------|
| Agricultural  | 12.05               | 0.1066     |
| Individual    | 10.44               | 0.0987     |
| SME           | 10.92               | 0.1126     |
| Women-owned   | 10.41               | 0.0889     |

**Interpretation**: Agricultural loans yield the highest return (12.05%) but carry elevated default risk (10.66%), attributable to sector volatilities such as environmental factors. SME loans exhibit moderate yield (10.92%) with the highest PD (11.26%), signaling potential operational risks in small enterprises. Women-owned loans demonstrate the lowest PD (8.89%) and competitive yield (10.41%), underscoring their alignment with SDG gender equality goals and suitability for impact-optimized portfolios. These metrics suggest opportunities for risk mitigation, such as collateral enhancements, to balance profitability and social returns.




In [14]:
# Define a function to compute Portfolio Yield for each subgroup.
def compute_yield(group):
    # (group['Interest_Rate'] * group['Utilization'] * group['Outstanding_Balance']): Element-wise multiplication (*) across Series (one-dimensional labeled arrays accessed via bracket notation); broadcasts operations row-wise.
    weighted_interest = (group['Interest_Rate'] * group['Utilization'] * group['Outstanding_Balance']).sum()  
    total_balance = group['Outstanding_Balance'].sum()  # Aggregates the denominator for normalization.
    yield_pct = (weighted_interest / total_balance * 100) if total_balance > 0 else 0  # Scalar division and multiplication; conditional (if-else) prevents errors in empty subgroups.
    return yield_pct  # Returns the percentage as a float.

# Define a function to compute Average PD for each subgroup.
# This function averages the pre-computed PD values.
def compute_pd(group):
    return group['PD'].mean()  

# Group the DataFrame by Borrower_Type and apply the functions.
yield_by_type = df.groupby('Borrower_Type').apply(compute_yield, include_groups=False).round(2)  # .round(2): A method on the resulting Series that limits decimals to two places.
pd_by_type = df.groupby('Borrower_Type').apply(compute_pd, include_groups=False).round(4)  # Rounded to four decimals for precision in probabilities.

# Construct an output DataFrame for structured presentation.
result = pd.DataFrame({
    'Borrower_Type': yield_by_type.index,  
    'Portfolio Yield (%)': yield_by_type.values,  
})  # Combines both Series for a comprehensive table.
print(result.to_string(index=False))  

Borrower_Type  Portfolio Yield (%)
 Agricultural                12.05
   Individual                10.44
          SME                10.92
  Women-owned                10.41


### Loss Given Default in Microfinance Portfolio Analysis

In the realm of credit risk management within microfinance, Loss Given Default (LGD) represents the estimated proportion of a loan's exposure that remains unrecovered following a borrower's default, after accounting for recoveries such as collateral liquidation or insurance proceeds. Expressed as a decimal (ranging from 0 to 1) or percentage, LGD is a pivotal component in calculating expected credit losses (ECL) and provisioning requirements, directly influencing profitability assessments and capital utilization in impact investment portfolios. Analyzing LGD supports the preparation of detailed risk reports and financial models, particularly in evaluating mitigation factors like collateral in high-risk emerging market private debt, such as microfinance loans tied to agricultural or clean energy projects.

In this dataset ("credit_risk_portfolio.csv" with 500 loan records), LGD is pre-computed per loan, starting from a base LGD_Base of 0.45 (45%) and adjusted downward for secured loans (e.g., to 0.315 or 31.5% via a Collateral_Factor of 0.8). The analysis computes the average LGD by Borrower_Type (Agricultural, Individual, SME, Women-owned), providing subgroup insights into recovery potential.

#### Formula

The Average LGD is the arithmetic mean of individual LGD values within each subgroup:
$$\text{Average LGD} = \frac{\sum \text{LGD}}{n}$$
where:

- $\sum \text{LGD}$: The summation, denoted by $\sum$, aggregates the LGD values (each a decimal between 0 and 1) across all loans in the subgroup.
- $n$: The number of loans in the subgroup, normalizing the mean to reflect the subgroup's composition.

For completeness, the underlying per-loan LGD derivation is:
$$\text{LGD} = \text{LGD\_Base} \times (1 - \text{Collateral Adjustment})$$
where LGD_Base is the baseline loss rate (0.45), and Collateral Adjustment is a reduction factor (e.g., 0.3 if Secured=True, yielding LGD=0.315).



#### Results: Average LGD by Borrower Type
The computed averages, expressed as percentages for interpretive clarity (rounded to two decimal places), are as follows:

| Borrower_Type | Average LGD (%) |
|---------------|-----------------|
| Agricultural  | 39.50           |
| Individual    | 41.12           |
| SME           | 40.92           |
| Women-owned   | 40.30           |

**Interpretation**: Agricultural loans exhibit the lowest average LGD (39.50%), likely due to higher collateralization in sector-specific financing (e.g., equipment for clean cooking projects), enhancing recovery rates amid default risks. Individual loans show the highest LGD (41.12%), reflecting potentially unsecured personal exposures in emerging markets. SME and Women-owned segments fall in between (40.92% and 40.30%, respectively), with the latter benefiting from targeted impact structures that may include informal guarantees. These variations underscore the value of collateral in mitigation strategies, to optimize risk-adjusted returns while advancing SDG-aligned initiatives.



In [15]:
# Define a function to compute Average LGD for each subgroup.
# This custom function receives a subgroup DataFrame and returns the mean LGD value.
def compute_lgd(group):
    return group['LGD'].mean()  

lgd_by_type = df.groupby('Borrower_Type').apply(compute_lgd, include_groups=False).round(4)  
# Construct an output DataFrame for tabular display.

result = pd.DataFrame({
    'Borrower_Type': lgd_by_type.index,  
    'Average LGD': (lgd_by_type.values * 100).round(2)  
})  # Integrates the metrics into a cohesive table.
print(result.to_string(index=False))  

Borrower_Type  Average LGD
 Agricultural        39.50
   Individual        41.12
          SME        40.92
  Women-owned        40.30


### Exposure at Default in Microfinance Portfolio Analysis

In credit risk modeling for microfinance institutions, Exposure at Default (EAD) quantifies the total amount of credit exposure anticipated at the moment of a borrower's default, encompassing outstanding principal, accrued interest, and potential undrawn commitments. This metric is a cornerstone of expected credit loss (ECL) calculations, bridging the gap between current balances and potential future drawdowns, and is crucial for accurate provisioning and liquidity forecasting. EAD analysis facilitates the evaluation of capital utilization and risk-adjusted profitability in emerging market private debt portfolios, such as those supporting microfinance or access-to-energy initiatives, by highlighting exposure concentrations that could amplify losses under stress scenarios.

The provided "credit_risk_portfolio.csv" dataset (comprising 500 loan records from frontier markets) includes a pre-computed EAD column, derived from Outstanding_Balance scaled by Utilization (the drawn-down proportion of the loan commitment). The analysis computes the average EAD by Borrower_Type (Agricultural, Individual, SME, Women-owned), offering subgroup-level insights into exposure profiles.

#### Formula

The Average EAD is the arithmetic mean of individual EAD values within each subgroup:
$$\text{Average EAD} = \frac{\sum \text{EAD}}{n}$$
where:

- $\sum \text{EAD}$: The summation, denoted by $\sum$, aggregates the EAD values (each in monetary units, such as USD) across all loans in the subgroup.
- $n$: The number of loans in the subgroup, serving as the divisor to compute the mean exposure.

Per-loan EAD is typically approximated as:
$$\text{EAD} = \text{Outstanding\_Balance} \times (1 + \text{Utilization Adjustment})$$
where Utilization Adjustment accounts for potential future draws (e.g., based on the Utilization ratio, a scalar from 0 to 1 representing the fraction of the commitment drawn).



#### Results: Average EAD by Borrower Type
The computed averages, rounded to two decimal places for reporting accuracy, are presented below:

| Borrower_Type | Average EAD |
|---------------|-------------|
| Agricultural  | 6178.11     |
| Individual    | 5999.11     |
| SME           | 5367.08     |
| Women-owned   | 5433.73     |

**Interpretation**: Agricultural loans display the highest average EAD (6178.11), reflecting larger commitment sizes potentially tied to seasonal or project-based financing, which heightens systemic risks in clean cooking or energy access portfolios amid environmental uncertainties. Individual loans follow closely (5999.11), indicative of personal borrowing scales in emerging markets. SME exposures are the lowest (5367.08), suggesting more granular lending structures, while Women-owned loans (5433.73) balance moderate size with impact focus. These patterns advocate for diversified allocation strategies to mitigate concentration risks, ensuring alignment with sustainable development goals while preserving liquidity.



In [16]:
# Define a function to compute Average EAD for each subgroup.
# This user-defined function accepts a subgroup DataFrame and returns the mean EAD value.
def compute_ead(group):
    return group['EAD'].mean()  

ead_by_type = df.groupby('Borrower_Type').apply(compute_ead, include_groups=False).round(2)  # .round(2): A method invoked on the resultant Series (a one-dimensional entity with .index for group identifiers and .values for numeric outcomes) that truncates values to two decimal places for succinct presentation.

# Construct an output DataFrame for tabular formatting.

result = pd.DataFrame({
    'Borrower_Type': ead_by_type.index,  
    'Average EAD': ead_by_type.values  
})  
print(result.to_string(index=False))  

Borrower_Type  Average EAD
 Agricultural      6178.11
   Individual      5999.11
          SME      5367.08
  Women-owned      5433.73
